<center> <h1>SEIR model</h1> </center>


SEIR model is one of the compartmental models which is used in mathematical modelling of infectious diseases. Here, the population is assigned to compartments with labels:
- S : Susceptible 
- E : Exposed 
- I : Infectious
- R : Recovered 

S is the fraction people who are able to contract the disease. E is the fraction of people who have been infected but are not yet infectious. I is the fraction of people who are capable of transmitting the disease. R is the fraction of people who have been recovered and have become immune. 

The number of people in S, E, I, R combine to form the population N. 
- $S + E + I + R = N$

Model:  
\begin{align*}
\mathrm{S} \overset{\beta }{\longrightarrow} \mathrm{E} \overset{\gamma}{\longrightarrow} \mathrm{I} \overset{\alpha}{\longrightarrow} \mathrm{R}  \\
\end{align*}

The infectious rate, \mathrm{\beta}, controls the rate of spread which represents the probability of transmitting disease between a susceptible and an exposed individual. The incubation rate,\mathrm{\gamma}, is the rate of latent individuals becoming infectious (average duration of incubation is 1/\mathrm{\gamma}). Recovery rate, \mathrm{\alpha} is the rate at which people recover.

Ordinary Differential Equation (ODE):  
\begin{align*}
& \frac{\mathrm{d}S}{\mathrm{d}T}= - N^{-1}\beta S I  \\
& \frac{\mathrm{d}I}{\mathrm{d}T}= N^{-1}\beta S I - \gamma E  \\
& \frac{\mathrm{d}I}{\mathrm{d}T}= -\gamma E - \alpha I  \\
& \frac{\mathrm{d}R}{\mathrm{d}T}= \alpha I  \\
\end{align*}




**Background**: When you first get infected with a disease often you might not have symptoms and may not be spreading. This is the latency period. 

**Importing necessary libraries**:
1. Numpy: NumPy offers comprehensive mathematical functions, random number generators, linear algebra routines, Fourier transforms, and more. [Link](https://numpy.org/)
2. scipy: SciPy provides algorithms for optimization, integration, interpolation, eigenvalue problems, algebraic equations, differential equations, statistics and many more. [Link](https://scipy.org/)
3. scipy.integrate: The scipy.integrate sub-package provides several integration techniques including an ordinary differential equation integrator. [Link](https://docs.scipy.org/doc/scipy/tutorial/integrate.html)
4. matplotlib: Matplotlib is a comprehensive library for creating static, animated, and interactive visualizations in Python. [Link](https://matplotlib.org/)
5. scipy.optimize: SciPy optimize provides functions for minimizing (or maximizing) objective functions, possibly subject to constraints.[Link](https://docs.scipy.org/doc/scipy/reference/optimize.html)

In [1]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from scipy.optimize import minimize
%matplotlib inline

Here N is the total population which is 1000. E0 which are the initially exposed which are 0. We have 10 infected people I0 and 990 susceptible people S0. 

We see that, S + E + I + R =  990 + 0 + 10 + 0 = 1000 = N, which is the total population. 

We have assumed beta which is the 

In [3]:
NumberofIterations = 10
N = 1000              # total population
E0 = 0                # Exposed
I0 = 10              # Infectious
R0 = 0                # Recovered
S0 = 999              # Susceptible

# S0 = N - I0 - E0 - R0
init_cond = S0, E0, I0, R0 # Initial conditions
Fitted_Parameters = 0.0001, 0.2, 0.03 # Fitted parameters (beta, gamma and alpha)
Initial_Guess = 0.0001, 0.2, 0.03    # Initial guess for our parameters

# Transition Rates
# beta = 0.5            # Disease transmission between Susceptible and exposed
# gamma = 0.3           # Disease transmission between exposed and infectious
# alpha = 0.3           # Recovery rate

# Calculating over the course of one year
daily = np.linspace(0, 365, 365)  # Returns evenly spaced numbers over an interval 0 to 365
tspan = daily

# Convering the ode model to a function
# The function SEIR_model takes in y, t, N, beta, gamma and alpha as it's inputs
    # y is the array of initial conditions - [S0, R0, I0, R0]
    # t is the array of time
    # N is the total population 
    # beta, gamma and alpha are the transition rates
    

# Initializing all of our parameters
Number_Parameters = len(Fitted_Parameters)   # Number of parameters 
ARE = np.zeros((6 , Number_Parameters), dtype=float) # Storage for Average Relative Estimation error
EstiParam = np.zeros((Number_Parameters,NumberofIterations, 6), dtype = float) # Storage for estimated parameters
EstiParams = np.zeros((Number_Parameters,NumberofIterations), dtype = float) # Storage for estimated parameters
InfectData = np.zeros((len(tspan),NumberofIterations, 6), dtype = float)
ExitFlag = np.zeros((1,NumberofIterations, 6), dtype = int) # Storage for exitflag
Fval = np.zeros((1,NumberofIterations, 6), dtype = float) # Storage for optimization values 
Levels = np.array([0, 0.01, 0.05, 0.1, 0.2, 0.3]) # Noise levels from paper

# This function returns the system of ode's which we will solve using the odeint function
def SEIR_model(y, t, beta, gamma, alpha):
    S, E, I, R = y
    dSdt = -beta*S*I             #dS
    dEdt = (beta*S*I) - gamma*E  #dE
    dIdt = gamma*E - alpha*I          #dI
    dRdt = alpha*I                    #dR
    return dSdt, dEdt, dIdt, dRdt

y_est = odeint(SEIR_model, init_cond, daily, args = Fitted_Parameters)

In [4]:
# The function err_in_dataSEIR takes in par which are the SEIR_model arguments and the optimized data
# The function returns the error 
def err_in_dataSEIR(par, odata):
    N = 1000              # total population
    E0 = 0                # Exposed
    I0 = 10               # Infectious
    R0 = 0                # Recovered
    S0 = N - I0 - E0 - R0 # Susceptible
    init_cond = [S0, E0, I0, R0] # Array of intial conditions (number of susceptible, exposed, infected and recovered people)
    
    EndofTime = 365       # Yearly    
    daily = np.linspace(1, EndofTime, EndofTime) #  Returns evenly spaced numbers over an interval 0 to 365
    tspan = daily 
    
    y = odeint(SEIR_model, init_cond, daily, args = (par[0],par[1],par[2]))
    # Here we are solving the system of equations that we modelled earlier
    # SEIR_model is the function that returns derivative values at requested y and t values as dydt
    # init_cond are the initial conditions of the differential states
    # t is the time points at which the solutions should be reported
    # arguments are the input values that our model needs (extra arguments to pass to the function)
    
    Infected = y[:,2] # second column of the solution corresponding to infected
    weight = 1/np.square(Infected)
    error_in_data = np.sum(weight*(Infected - odata)**2) # finding the error
    
    return error_in_data
    
    

In [5]:

for IterationLevels in range(6):
    NoiseLevel = Levels[IterationLevels] # Defines noise level for current iteration
    for i in range(NumberofIterations):
        Noise = NoiseLevel*y_est[:,2].T
        Prevalencedata = np.random.normal(y_est[:,2].T, Noise, 365)  
# Equation from step 2. Section 4.1. Use normal distribution. 

        # Finding the sum of "value"
        value = 0
        for j in Prevalencedata:
            if j < 0:
                value = value + 1
        # If the sum of these are non-zero, we enter the while-loop. Otherwise, proceed as normal. 
        while value != 0:
            Prevalencedata = np.random.normal(y_est[:,2].T, Noise, 365).T
            value = 0
            for k in Prevalencedata:
                if k < 0:
                    value = value + 1
                    
        #Lowerbounds = np.array([0,0,0])
        #Upperbounds = np.array([1, 1, 1])
        # Bounds for the parameters
        bnds = ((0,1),(0,1),(0,1))
        
        # Optimization
        optimizer0 = minimize(err_in_dataSEIR, Initial_Guess, args=Prevalencedata , method='Nelder-Mead')
        EstimatedParameters = optimizer0.x
        print(EstimatedParameters.T)
        exitflag = optimizer0.success
        fval = optimizer0.fun
        message = optimizer0.message
        if exitflag==0:
            print(message)
        
        # value at optimal point - fval 
        # whether the function - convergence rate - exitflag 

        EstiParams[:,i] = EstimatedParameters.T # Stores parameters for current noise level to computer ARE.  
        EstiParam[:, i, IterationLevels] = EstimatedParameters.T # Stores for each noise level
        InfectData[:, i, IterationLevels] = Prevalencedata.T
        ExitFlag[:, i, IterationLevels] = exitflag # Stores exitflags
        Fval[:, i, IterationLevels] = fval
    
    # Computing the ARE score
    ARE_Value = np.array(np.zeros((1, Number_Parameters), dtype = float))
 
    
    for i in range(Number_Parameters):
        ARE_Value[0,i] = (100/NumberofIterations) * np.sum(abs(Fitted_Parameters[i] - EstiParams[i,:]))/ abs(Fitted_Parameters[i])
    ARE[IterationLevels,:] = ARE_Value 


[1.01895336e-04 1.93281699e-01 2.98803748e-02]
[1.01895336e-04 1.93281699e-01 2.98803748e-02]
[1.01895336e-04 1.93281699e-01 2.98803748e-02]
[1.01895336e-04 1.93281699e-01 2.98803748e-02]
[1.01895336e-04 1.93281699e-01 2.98803748e-02]
[1.01895336e-04 1.93281699e-01 2.98803748e-02]
[1.01895336e-04 1.93281699e-01 2.98803748e-02]
[1.01895336e-04 1.93281699e-01 2.98803748e-02]
[1.01895336e-04 1.93281699e-01 2.98803748e-02]
[1.01895336e-04 1.93281699e-01 2.98803748e-02]
[1.02118365e-04 1.91234316e-01 2.98676796e-02]
[1.02010066e-04 1.92496857e-01 2.98591452e-02]
[1.01902086e-04 1.92935382e-01 2.98856369e-02]
[1.01682789e-04 1.94880888e-01 2.98878463e-02]
[1.01809378e-04 1.93877373e-01 2.98961466e-02]
[1.02066507e-04 1.91768294e-01 2.98633761e-02]
[1.01975240e-04 1.92333898e-01 2.98817788e-02]
[1.01983781e-04 1.92599151e-01 2.98751525e-02]
[1.01725956e-04 1.94714187e-01 2.98783485e-02]
[1.01882411e-04 1.93724574e-01 2.98621982e-02]
[1.01770520e-04 1.92869834e-01 2.99333295e-02]
[1.03429485e-

In [6]:
ARE

array([[ 1.89533558,  3.35915055,  0.39875077],
       [ 1.915658  ,  3.47175411,  0.41423047],
       [ 2.06616577,  4.87092435,  0.60366508],
       [ 2.8202408 ,  8.70533506,  1.22042814],
       [ 4.83096767, 13.26465056,  3.36836852],
       [19.44943072, 27.80901492,  7.41670504]])